In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
path_to_csv = '/content/gdrive/My Drive/Sentiment.csv'


Mounted at /content/gdrive


In [4]:
import pandas as pd
dset = pd.read_csv(path_to_csv, header=0)
mask = dset.columns.isin(['text', 'sentiment'])
data = dset.loc[:, mask]
data['text'] = data['text'].apply(lambda x: x.lower())                              # Keeping only the necessary columns
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

<ipython-input-4-426e98980e14>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())                              # Keeping only the necessary columns
<ipython-input-4-426e98980e14>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))


In [5]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X) #Padding the feature matrix

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential() #Sequential Neural Network
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))     #input dimension 2000 Neurons, output dimension 128 Neurons
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) #Compiling the model
    return model
# print(model.summary())

In [6]:
label_encoder = LabelEncoder() #Applying label Encoding on the label matrix
integer_encoded = label_encoder.fit_transform(data['sentiment']) #fitting the model
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()                                                #Function call to Sequential Neural Network
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)         #evaluating the model
print(score)
print(acc)

291/291 - 61s - loss: 0.8245 - accuracy: 0.6424 - 61s/epoch - 210ms/step
144/144 - 3s - loss: 0.7612 - accuracy: 0.6769 - 3s/epoch - 23ms/step
0.7611876726150513
0.6769331693649292


In [7]:
print(model.metrics_names)


['loss', 'accuracy']


In [8]:
#question 1:
model.save('sentimentAnalysis.h5') #Saving the model


In [9]:
from keras.models import load_model
model= load_model('sentimentAnalysis.h5')                #loading the saved model
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [10]:
# Predicting on the text data
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence)
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0)
sentiment_probs = model.predict(sentence, batch_size=1, verbose=2)[0]
sentiment = np.argmax(sentiment_probs)

print(sentiment_probs)
if sentiment == 0:
    print("Neutral")
elif sentiment < 0:
    print("Negative")
elif sentiment > 0:
    print("Positive")
else:
    print("Cannot be determined")


1/1 - 0s - 324ms/epoch - 324ms/step
[0.40014437 0.10924453 0.4906111 ]
Positive


In [11]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
model = KerasClassifier(build_fn=createmodel,verbose=2)
batch_size= [10, 20, 40]
epochs = [1, 2]
param_grid= {'batch_size':batch_size, 'epochs':epochs} #creating dictionary for batch size, no. of epochs
grid  = GridSearchCV(estimator=model, param_grid=param_grid) #Applying dictionary with
grid_result= grid.fit(X_train,Y_train) #Fitting the model

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #best score, best hyper parameters

<ipython-input-11-6c99b49150f4>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters


744/744 - 118s - loss: 0.8166 - accuracy: 0.6528 - 118s/epoch - 159ms/step
186/186 - 3s - loss: 0.7420 - accuracy: 0.6832 - 3s/epoch - 15ms/step
744/744 - 117s - loss: 0.8239 - accuracy: 0.6473 - 117s/epoch - 157ms/step
186/186 - 7s - loss: 0.7588 - accuracy: 0.6783 - 7s/epoch - 37ms/step
744/744 - 118s - loss: 0.8230 - accuracy: 0.6446 - 118s/epoch - 158ms/step
186/186 - 3s - loss: 0.7524 - accuracy: 0.6794 - 3s/epoch - 15ms/step
744/744 - 118s - loss: 0.8264 - accuracy: 0.6447 - 118s/epoch - 159ms/step
186/186 - 5s - loss: 0.7491 - accuracy: 0.6825 - 5s/epoch - 27ms/step
744/744 - 118s - loss: 0.8164 - accuracy: 0.6514 - 118s/epoch - 159ms/step
186/186 - 3s - loss: 0.7746 - accuracy: 0.6679 - 3s/epoch - 15ms/step
Epoch 1/2
744/744 - 119s - loss: 0.8276 - accuracy: 0.6496 - 119s/epoch - 160ms/step
Epoch 2/2
744/744 - 118s - loss: 0.6815 - accuracy: 0.7135 - 118s/epoch - 158ms/step
186/186 - 3s - loss: 0.7354 - accuracy: 0.6762 - 3s/epoch - 16ms/step
Epoch 1/2
744/744 - 117s - loss: 0.